In [112]:
import open3d as o3d
import numpy as np

def main():
    # Load the PLY file
    bunny = o3d.io.read_point_cloud("C:\\Users\\adamt\\OneDrive - Syddansk Universitet\\2. semester\\Førsteårsprojekt\\bunny\\bunny\\reconstruction\\bun_zipper.ply")
    # Print some info
    bunny2 = o3d.io.read_point_cloud("C:\\Users\\adamt\\OneDrive - Syddansk Universitet\\2. semester\\Førsteårsprojekt\\bunny\\bunny\\reconstruction\\bun_zipper.ply")
    translation = np.multiply(-1,o3d.geometry.PointCloud.get_center(bunny))
    o3d.geometry.PointCloud.translate(bunny,translation)
    o3d.geometry.PointCloud.translate(bunny2,translation)
    #bunny_subset = o3d.io.read_point_cloud("C:\\Users\\adamt\\OneDrive - Syddansk Universitet\\2. semester\\Førsteårsprojekt\\bunny\\bunny\\data\\bun000.ply")
    #translation2 = np.multiply(-1,o3d.geometry.PointCloud.get_center(bunny_subset))
    #o3d.geometry.PointCloud.translate(bunny_subset,translation2)
    # Define a transformation matrix (4x4)
    # Example: rotate 45 degrees around Z 
    theta = np.pi / 4  # 45 degrees
    transformation = np.array([
        [np.cos(theta), -np.sin(theta), 0, 0],
        [np.sin(theta),  np.cos(theta), 0, 0],
        [0,              0,             1, 0],
        [0,              0,             0, 1]
    ])

    # Apply transformation
    bunny2.transform(transformation)
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.001)
    
    # Visualize the point cloud
    o3d.visualization.draw_geometries([bunny,bunny2,sphere])

    #o3d.io.write_point_cloud(r"C:\Users\adamt\OneDrive - Syddansk Universitet\2. semester\Førsteårsprojekt\bunny\bun_bin.ply", bunny, write_ascii=False)

    #o3d.io.write_point_cloud(r"C:\Users\adamt\OneDrive - Syddansk Universitet\2. semester\Førsteårsprojekt\bunny\bun2_bin.ply", bunny2, write_ascii=False)
    o3d.geometry.PointCloud.paint_uniform_color(bunny,[0,1,0])
    o3d.geometry.PointCloud.paint_uniform_color(bunny2,[1,0,0])
    trans = np.array([
        [0.70702475, -0.70718879,  0.00013018,  0.00000382],
        [0.70718875,  0.70702476,  0.00026395,  0.00000647],
        [-0.0002787,  -0.00009456,  0.99999996,  0.00003825],
        [0.,          0.,          0.,          1.]])
    bunny.transform(trans)
    o3d.visualization.draw_geometries([bunny,bunny2,sphere])
    #import os
    #print(os.getcwd())


if __name__ == "__main__":
    main()
#results = o3d.io.read_point_cloud(r"C:\Users\adamt\data\results\bun_bin_on_bun2_bin_post_icp.ply")
#o3d.geometry.PointCloud.paint_uniform_color(results,[1,0,0])
#results2 = o3d.io.read_point_cloud(r"C:\Users\adamt\data\results\bun_bin_on_bun2_bin_post_ransac.ply")
#o3d.geometry.PointCloud.paint_uniform_color(results,[0,1,0])
#o3d.visualization.draw_geometries([results2])
#o3d.visualization.draw_geometries([results])

In [46]:
import os
from typing import Union
import numpy as np
import open3d as o3d
from glob import glob
from open3d.geometry import PointCloud

def make_directory(directory_path: str) -> None:
    """Creates a directory if it does not exist."""
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

def load_point_cloud(file_or_directory_path: str, file_extension: str = 'ply') -> Union[np.ndarray, PointCloud]:
    """
    Loads a point cloud file or all point cloud files in a directory.

    Args:
        file_or_directory_path (str): Path to a point cloud file or directory.
        file_extension (str): Extension of the point cloud files (default is 'ply').

    Returns:
        Union[np.ndarray, PointCloud]: The loaded point cloud(s).
    """
    if os.path.isdir(file_or_directory_path):
        point_clouds = []
        point_cloud_files = glob(f"{file_or_directory_path}/*.{file_extension}")
        for file_path in point_cloud_files:
            point_cloud = o3d.io.read_point_cloud(file_path, format=file_extension)
            point_clouds.append(point_cloud)
        return np.array(point_clouds)
    
    elif os.path.isfile(file_or_directory_path):
        return o3d.io.read_point_cloud(file_or_directory_path, format=file_extension)

def convert_point_cloud_to_bin(point_cloud: PointCloud, output_file_path: str, file_extension: str = 'ply') -> None:
    """
    Converts a point cloud to binary format (.bin).

    Args:
        point_cloud (Union[str, PointCloud, np.ndarray]): The point cloud or its path.
        output_file_path (str): Path for the output .bin file.
    """
    if isinstance(point_cloud, str):
        point_cloud = load_point_cloud(file_or_directory_path=point_cloud, file_extension=file_extension)
    
    output_file_path += "" if output_file_path.endswith(".bin") else ".bin"
    np.asarray(point_cloud.points).astype('float32').tofile(output_file_path)

def batch_convert_point_clouds_to_bin(input_directory_path: str, output_directory_path: str, file_extension: str = 'ply') -> None:
    """
    Converts all point clouds in a directory from their current format to .bin format.

    Args:
        input_directory_path (str): Path to the directory containing point cloud files.
        output_directory_path (str): Path to the directory where .bin files will be saved.

    Raises:
        ValueError: If input_directory_path is neither a file nor a directory.
    """
    if not os.path.isdir(output_directory_path):
        make_directory(output_directory_path)

    if os.path.isfile(input_directory_path):
        file_name_without_extension = os.path.basename(input_directory_path).split('.')[0]
        output_file_path = os.path.join(output_directory_path, file_name_without_extension)
        convert_point_cloud_to_bin(point_cloud=input_directory_path, output_file_path=output_file_path)
    
    elif os.path.isdir(input_directory_path):
        point_cloud_files = glob(f"{input_directory_path}/*.{file_extension}")
        for file_path in point_cloud_files:
            file_name_without_extension = os.path.basename(file_path).split('.')[0]
            output_file_path = os.path.join(output_directory_path, file_name_without_extension)
            convert_point_cloud_to_bin(point_cloud=file_path, output_file_path=output_file_path)
    
    else:
        raise ValueError(f"'{input_directory_path}' is neither a file nor a directory.")

In [48]:
#convert_point_cloud_to_bin(load_point_cloud("C:\\Users\\adamt\\OneDrive - Syddansk Universitet\\2. semester\\Førsteårsprojekt\\bunny\\bunny\\data\\bun000.ply"),"C:\\Users\\adamt\\OneDrive - Syddansk Universitet\\2. semester\\Førsteårsprojekt\\bunny\\")

In [38]:
import open3d as o3d

# Load your ASCII .ply file
pcd = o3d.io.read_point_cloud(r"C:\Users\adamt\OneDrive - Syddansk Universitet\2. semester\Førsteårsprojekt\bunny\bunny\data\bun000.ply")

# Save it as binary .ply
o3d.io.write_point_cloud(r"C:\Users\adamt\OneDrive - Syddansk Universitet\2. semester\Førsteårsprojekt\bunny\bun000_bin.ply", pcd, write_ascii=False)


import os
print(os.getcwd())


C:\Users\adamt


NameError: name 'bunny' is not defined